In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gensim --quiet
!pip install altair_data_server --quiet
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
#import libraries
import pandas as pd
import os
from google.colab import drive

import numpy as np


import sklearn as sk
from sklearn.model_selection import train_test_split
import os

import matplotlib.pyplot as plt

import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

import transformers
import torch
transformers.logging.set_verbosity_error()
import altair as alt
from altair_data_server import data_server

from transformers import T5Tokenizer, TFT5Model, T5ForConditionalGeneration, TrainingArguments, Trainer, T5ForSequenceClassification
from datasets import Dataset, load_dataset


#Read Data

In [ ]:
#for sequence classification
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
train_filepath = '/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/Old Data/sentence_training_data.csv'
test_filepath = '/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/Old Data/sentence_testing_data.csv'

In [ ]:
train_dataset = pd.read_csv(train_filepath)
num_train_examples = len(train_dataset)

In [ ]:
#stream dataset
train_dataset = load_dataset('csv', data_files = train_filepath, streaming = True)['train']
test_dataset = load_dataset('csv', data_files = test_filepath, streaming = True)['train']

In [ ]:
for example in train_dataset:
  y = example['y']
  print(y)
  break

0


In [ ]:
def create_t5_tokenize(df_row, MAX_LENGTH = 256):
  """
  Using target as text instead of numerical testing: Need to switch to t5 conditional generator
  """
  X = df_row['doc_text']
  y = df_row['label']

  if y == 0:
    target_text = 'Not Important'
  else:
    target_text = 'Important'
  X_encoded = t5_tokenizer.batch_encode_plus(
      [f'binary classification: {X}'],
      max_length = MAX_LENGTH,
      padding = 'max_length',
      truncation = True,
      return_attention_mask = True,
      return_tensors = 'pt'
  )

  X_input_ids = X_encoded['input_ids'][0]
  X_attention_mask = X_encoded['attention_mask'][0]

  target_encoded = t5_tokenizer.batch_encode_plus(
      [target_text],
      max_length = MAX_LENGTH,
      padding = 'max_length',
      truncation = True,
      return_attention_mask = True,
      return_tensors = 'pt'
  )

  label_ids = target_encoded['input_ids'][0]

  return {'input_ids': X_input_ids,
          'attention_mask': X_attention_mask,
          'labels': label_ids}

In [ ]:
train_dataset = train_dataset.map(create_t5_tokenize)
test_dataset = test_dataset.map(create_t5_tokenize)

In [ ]:
#show example output
for example in train_dataset:
  print(example)
  break

{'X': 'You will be responsible for withholding, filing, and reporting all taxes, duties, and other governmental assessments associated with your activity in connection with the Services.', 'y': 0, 'input_ids': tensor([14865, 13774,    10,   148,    56,    36,  1966,    21,    28, 21647,
            6,  9479,     6,    11,  5099,    66,  5161,     6,  9353,     6,
           11,   119,     3, 18984, 16396,  1968,    28,    39,  1756,    16,
         2135,    28,     8,  1799,     5,     1,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
          

In [ ]:
batch_size = 5
num_epochs = 3

#save model for fine - tune
dir_path = '/content/drive/MyDrive/266 Final Project/model_checkpoints'
file_path = dir_path + 't5base-finetuned-seqclassification'

training_args = TrainingArguments(
    output_dir= file_path,          # output directory; file_path
    evaluation_strategy="epoch",     # evaluation strategy to adopt during training
    learning_rate=1e-5,              # learning rate
    per_device_train_batch_size=batch_size,   # batch size for training
    per_device_eval_batch_size=batch_size,    # batch size for evaluation
    max_steps = int(num_epochs * num_train_examples / batch_size),
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    report_to="none"        # Important, otherwise will try to connect and write to wandb
)

trainer = Trainer(model=t5_model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)

trainer.train()

results = trainer.evaluate(eval_dataset = test_dataset) #pass in test data?
print(results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 1.4644, 'grad_norm': 0.05320454761385918, 'learning_rate': 8.607630186577556e-06, 'epoch': 0.1392369813422445}
{'loss': 0.0079, 'grad_norm': 0.011275972239673138, 'learning_rate': 7.215260373155111e-06, 'epoch': 0.278473962684489}
{'eval_loss': 0.045644648373126984, 'eval_runtime': 372.5796, 'eval_samples_per_second': 25.82, 'eval_steps_per_second': 5.164, 'epoch': 0.3336118072960178}
{'loss': 0.0022, 'grad_norm': 0.050161704421043396, 'learning_rate': 5.822890559732666e-06, 'epoch': 1.0840991367307158}
{'loss': 0.002, 'grad_norm': 0.038872648030519485, 'learning_rate': 4.43052074631022e-06, 'epoch': 1.22333611807296}
{'eval_loss': 0.03983098268508911, 'eval_runtime': 370.8478, 'eval_samples_per_second': 25.941, 'eval_steps_per_second': 5.188, 'epoch': 1.333611807296018}
{'loss': 0.0019, 'grad_norm': 0.04135437309741974, 'learning_rate': 3.038150932887775e-06, 'epoch': 2.028961292119187}
{'loss': 0.0006, 'grad_norm': 0.16463729739189148, 'learning_rate': 1.64578111946533e-06, 

In [ ]:
test_data_df = pd.read_csv(test_filepath)
train_data_df = pd.read_csv(train_filepath)
print(test_data_df)

      service_id                                           doc_text highlight  \
0           2529  You agree to provide current, complete, and ac...       NaN   
1           1638  Redbubble will also charge the customer for th...       NaN   
2           2416  Bitdefender Total Security 2016 Protected by U.S.       NaN   
3            356  Because user authentication on the Internet is...       NaN   
4            315  and\nd Where relevant, for compliance with leg...       NaN   
...          ...                                                ...       ...   
9615         177                              Thats our commitment.       NaN   
9616        1731  Also, remember to log off the FollowMyHealth s...       NaN   
9617        1731  Or write to the company at\n Allscripts Privac...       NaN   
9618        1012  Enable solely internal uses that are reasonabl...       NaN   
9619         225  Youll always have the option to change your mi...       NaN   

      label  
0       0.0  

In [ ]:
print(test_data_df[test_data_df['y'] == 0]['X'][0])

Because we are subject to legal and regulatory requirements relating to drivers and delivery persons, this generally means that we retain their account and data for a minimum of 7 years after a deletion request.


In [ ]:
#TESTING FOR USING TEXT GENERATION
for test_input_text in ['binary classification: Your violation of the law of any relevant jurisdiction\nor any claim that use of Your data caused damage to a third party.',
                        'binary classification: As part of that sale or transfer, we may pass information we have collected and stored, including personal information, to anyone involved in the sale or transfer.',
                        'binary classification: This Privacy Policy may be amended from time to time, and if we make material changes to it, we will provide you notice through the Services, or by other appropriate means including but not limited to email, via this page or via the user interface.',
                        'binary classification: Sellers must deliver the same quality of service as shown on their Gig images.',
                        'binary classifcation: Because we are subject to legal and regulatory requirements relating to drivers and delivery persons, this generally means that we retain their account and data for a minimum of 7 years after a deletion request.']:
    test_inputs = t5_tokenizer([test_input_text], return_tensors='pt')
    test_output_ids = t5_model.generate(test_inputs['input_ids'].cuda())
    print([t5_tokenizer.decode(out_ids, skip_special_tokens = True, clean_up_tokenization_spaces = False) for out_ids in test_output_ids])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Important']
['Important']
['Important']
['Important']
['Important']


In [ ]:
results_path = '/content/drive/MyDrive/266 Final Project/Processed Data/sentence_t5_test.csv'

def classify_t5_summary(text):
    transformers.logging.set_verbosity_error()
    # Tokenize the input text
    test_inputs = t5_tokenizer([f'binary classification: {text}'], return_tensors='pt')
    test_output_ids = t5_model.generate(test_inputs['input_ids'].cuda())
    return [t5_tokenizer.decode(out_ids, skip_special_tokens = True, clean_up_tokenization_spaces = False) for out_ids in test_output_ids]


#sum(X_test['t5_extract_flag']) / len(X_test)
#test_data_df.to_csv(results_path, index = False)

In [ ]:
#load checkpoint
dir_path = '/content/drive/MyDrive/266 Final Project/model_checkpoints'
file_path = dir_path + 't5base-finetuned-seqclassification'
t5_model = T5ForConditionalGeneration.from_pretrained(file_path + '/checkpoint-3789') #3789 old
t5_model = t5_model.to('cuda')

In [ ]:
# Test T5 summary generation
test_data_df['pred'] = test_data_df['doc_text'].apply(lambda x: classify_t5_summary(x))
train_data_df['pred'] = train_data_df['doc_text'].apply(lambda x: classify_t5_summary(x))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
train_data_df

,service_id,doc_text,highlight,label,pred
0,614,Cookies are widely used because they allow a w...,NaN,0.0,[Important]
1,403,You release us from liability concerning 5.2.1...,NaN,0.0,[Important]
2,186,"Permissions section of your Account Settings, ...",NaN,0.0,[Not Important]
3,591,"You acknowledge that you have only a limited, ...",NaN,0.0,[Important]
4,1209,11.0 CONFIDENTIAL INFORMATION\n\n11.1 Confiden...,NaN,0.0,[Important]
...,...,...,...,...,...
5981,2315,"For instance, if you enter a contest, promotio...","For instance, if you enter a contest, promotio...",1.0,[Important]
5982,1741,"and\nMeasure the placement, frequency, efficac...","and\nMeasure the placement, frequency, efficac...",1.0,[Not Important]
5983,314,"Pixels, web beacons, clear GIFs\n.","iPixels, web beacons, clear GIFs\n.",1.0,[Not Important]
5984,445,or\nUse features of our websites and mobile se...,or\nUse features of our websites and mobile se...,1.0,[Important]


In [ ]:
import ast
#train_data_df['pred'] = train_data_df['pred'].apply(ast.literal_eval)
train_data_df['pred'] = train_data_df['pred'].apply(lambda x: x[0])
train_data_df.loc[train_data_df['pred'] == 'Important', 'pred'] = 1
train_data_df.loc[train_data_df['pred'] == 'Not Important', 'pred'] = 0
train_data_df['pred'] = train_data_df['pred'].astype(int)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#accuracy metrics
print(len(train_data_df['label'] == 0))
print(len(train_data_df['pred'] == 0))
print(len(train_data_df['label'] == 1))
print(len(train_data_df['pred'] == 1))
train_y_true = train_data_df['label'].values
train_y_pred = train_data_df['pred'].values

accuracy = accuracy_score(train_y_true, train_y_pred)
print("Accuracy:", accuracy)

5986
5986
5986
5986
Accuracy: 0.5701637153357835


In [ ]:
train_data_df['pred'].value_counts()

,count
pred,
1,4816
0,1170


In [ ]:
import ast
#test_data_df = pd.read_csv(results_path)
#test_data_df['pred'] = test_data_df['pred'].apply(ast.literal_eval)


In [ ]:
test_data_df

In [ ]:
test_data_df['label'].value_counts()

,count
label,
0.0,8872
1.0,748


In [ ]:
test_data_df['pred'].value_counts()

,count
pred,
[Important],7215
[Not Important],2405


In [ ]:
#test_data_df['pred'] = test_data_df['pred'].apply(lambda x: x[0])
test_data_df.loc[test_data_df['pred'] == 'Important', 'pred'] = 1
test_data_df.loc[test_data_df['pred'] == 'Not Important', 'pred'] = 0
test_data_df['pred'] = test_data_df['pred'].astype(int)

In [ ]:
test_data_df['pred'].value_counts()

,count
pred,
1,7215
0,2405


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#accuracy metrics
print(len(test_data_df['label'] == 0))
print(len(test_data_df['pred'] == 0))
print(len(test_data_df['label'] == 1))
print(len(test_data_df['pred'] == 1))
test_y_true = test_data_df['label'].values
test_y_pred = test_data_df['pred'].values

accuracy = accuracy_score(test_y_true, test_y_pred)
precision = precision_score(test_y_true, test_y_pred, average='macro', zero_division=0)
recall = recall_score(test_y_true, test_y_pred, average='macro', zero_division=0)
f1 = f1_score(test_y_true, test_y_pred, average='macro', zero_division=0)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

9620
9620
9620
9620
Accuracy: 0.3057172557172557
Precision: 0.5224532224532225
Recall: 0.5587093083811112
F1 Score: 0.28448915731578495


In [ ]:
train_data_df['label'].value_counts()

,count
label,
0.0,2993
1.0,2993


In [ ]:
print(len(train_data_df + test_data_df))

9620
